In [2]:
from gibbs.dataclass import GibbsResult,get_results
import matplotlib.pyplot as plt
import numpy as np
import plotly_express as px
from gibbs.learning.bayesian_learning import BayesianLearning
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory
from qiskit.quantum_info import Statevector
from gibbs.learning.klocal_pauli_basis import KLocalPauliBasis
from scipy.linalg import block_diag
from scipy.sparse import bmat 
from qiskit.quantum_info import state_fidelity,Statevector
from gibbs.utils import number_of_elements, simple_purify_hamiltonian, spectral_dec,printarray, expected_state,identity_purification
from gibbs.plots.plots import preparation, compare_preparations, preparation_error
# plt.rcParams['text.usetex'] = True

%load_ext autoreload
%autoreload 2

gresults = get_results("../saved_simulations/turbo/random1localcfields")

In [3]:
get_results("../scripts_lap/job_12_mumbai")[0].betas

[0.0]

In [117]:
import scipy.stats as st
def get_best_distribution(data):
    dist_names = ["norm", "cauchy","expon","fatiguelife"]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)
        params[dist_name] = param
        # Applying the Kolmogorov-Smirnov test
        D, p = st.kstest(data, dist_name, args=param)
        print("p value for "+dist_name+" = "+str(p))
        dist_results.append((dist_name, p))

    # select the best fitted distribution
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    # store the name of the best fit and its p value

    print("Best fitting distribution: "+str(best_dist))
    print("Best p value: "+ str(best_p))
    print("Parameters for the best fit: "+ str(params[best_dist]))

    return best_dist, best_p, params[best_dist]

In [118]:
timestep = -1
errors = np.concatenate([np.real(g.cfaulties[timestep]-g.coriginal*g.betas[timestep]) for g in gresults])

In [136]:
timestep = -1
errors = np.concatenate([np.real(g.cfaulties[timestep]-g.coriginal*g.betas[timestep]) for g in gresults])
fig = px.histogram(x=[errors],histnorm="probability density",nbins=100)
fit = get_best_distribution(errors)
x = np.linspace(-0,0.04,10000)
y = getattr(st,fit[0])(*fit[2]).pdf(x)
y_test = st.expon.pdf(x,0,5e-3)
fig.add_scatter(x=x,y=y_test)
fig.show()

(5175,)
p value for norm = 0.0
p value for cauchy = 0.0
p value for expon = 0.0
p value for fatiguelife = 0.0
Best fitting distribution: norm
Best p value: 0.0
Parameters for the best fit: (0.009541056140807663, 0.041949620365408745)
